## Estudio optuna

In [67]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from sklearn.metrics import r2_score
import json

In [68]:
# Añadir pesos

def calculate_weights(target_y, features_X, base_high_w=10.0, alert_w=20.0, spike_w=40.0):
    weights = np.ones(len(target_y))
    pollen = target_y.values
    
    # Tier 1: Low-Medium Levels (>= 10)
    weights[pollen >= 10] = base_high_w
    
    # Tier 2: Warnings/Alerts (>= 50)
    weights[pollen >= 50] = alert_w
    
    # Tier 3: "Explosive Spike" (Jump >= 300)
    if 'polen_lag_1' in features_X.columns:
        pollen_today = pollen
        pollen_yesterday = features_X['polen_lag_1'].values
        try:
            jump = pollen_today - pollen_yesterday
            explosive_mask = (jump >= 300)
            weights[explosive_mask] = spike_w
        except:
            pass
            
    return weights


In [69]:
# Aplicar ruido de seguridad

def apply_safety_noise(X, seed=None):
    X_noisy = X.copy()
    
    NOISE_LEVEL = 0.20
    SAFETY_BIAS = 0.90
    
    if 'granos_de_polen_x_metro_cubico' in X_noisy.columns:
        noise = np.random.normal(SAFETY_BIAS, NOISE_LEVEL, size=len(X_noisy))
        X_noisy['granos_de_polen_x_metro_cubico'] = X_noisy['granos_de_polen_x_metro_cubico'] * noise
    
    return X_noisy

In [70]:
year = 2025

df = pd.read_csv(r"..\new_datasets\datos_gramineas.csv")
df['fecha'] = pd.to_datetime(df['fecha'])

FEATURES = json.load(open('json/features_t+1.json', 'r', encoding='utf-8'))
TARGET = 'granos_de_polen_x_metro_cubico'

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'objective': 'reg:squarederror',
        'random_state': 42
    }

    train_data = df[df['fecha'].dt.year < year].dropna(subset=[TARGET])
    test_data = df[df['fecha'].dt.year == year].dropna(subset=[TARGET])

    X_train = train_data[FEATURES]
    y_train = train_data[TARGET]

    X_test = test_data[FEATURES]
    y_test = test_data[TARGET]

    X_train = apply_safety_noise(X_train, seed=42)
    pesos = calculate_weights(y_train, X_train)

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100, sample_weight=pesos)

    preds = model.predict(X_test)
    return r2_score(y_test, preds)

In [71]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print(f"Mejor R2 encontrado: {study.best_value:0.2f}")
print("Mejores parámetros:", study.best_params)

with open('json/optuna_best_params_t+1.json', 'w', encoding='utf-8') as f:
    json.dump(study.best_params, f, ensure_ascii=False, indent=4)

[I 2026-02-21 20:55:32,875] A new study created in memory with name: no-name-ce3dd5ca-495d-47ec-b33c-0edc8711a454


[0]	validation_0-rmse:48.26830	validation_1-rmse:58.12041
[100]	validation_0-rmse:29.03962	validation_1-rmse:40.95143
[200]	validation_0-rmse:17.72027	validation_1-rmse:32.26031
[300]	validation_0-rmse:11.06276	validation_1-rmse:28.20404
[400]	validation_0-rmse:7.11829	validation_1-rmse:26.42071
[500]	validation_0-rmse:4.77825	validation_1-rmse:25.59308
[600]	validation_0-rmse:3.34625	validation_1-rmse:25.18265
[700]	validation_0-rmse:2.47248	validation_1-rmse:24.92573
[800]	validation_0-rmse:1.94128	validation_1-rmse:24.75964
[900]	validation_0-rmse:1.60186	validation_1-rmse:24.67947
[1000]	validation_0-rmse:1.37172	validation_1-rmse:24.61551
[1100]	validation_0-rmse:1.20974	validation_1-rmse:24.58240
[1200]	validation_0-rmse:1.09732	validation_1-rmse:24.56023
[1300]	validation_0-rmse:0.99928	validation_1-rmse:24.54531
[1400]	validation_0-rmse:0.90120	validation_1-rmse:24.53349
[1500]	validation_0-rmse:0.81621	validation_1-rmse:24.52800
[1600]	validation_0-rmse:0.74592	validation_1-rm

[I 2026-02-21 20:56:00,166] Trial 0 finished with value: 0.7519336313243744 and parameters: {'n_estimators': 1753, 'max_depth': 7, 'learning_rate': 0.0053605376106769164, 'subsample': 0.909640566371078, 'colsample_bytree': 0.9247702250648496, 'gamma': 1.5398686440575577}. Best is trial 0 with value: 0.7519336313243744.


[0]	validation_0-rmse:48.28069	validation_1-rmse:58.16111
[100]	validation_0-rmse:29.86152	validation_1-rmse:41.36235
[200]	validation_0-rmse:18.58874	validation_1-rmse:32.64410
[300]	validation_0-rmse:11.71588	validation_1-rmse:28.53745
[400]	validation_0-rmse:7.46575	validation_1-rmse:26.75056
[500]	validation_0-rmse:4.84481	validation_1-rmse:26.01763
[600]	validation_0-rmse:3.22287	validation_1-rmse:25.65840
[700]	validation_0-rmse:2.19819	validation_1-rmse:25.53432
[800]	validation_0-rmse:1.54619	validation_1-rmse:25.47747
[900]	validation_0-rmse:1.13644	validation_1-rmse:25.45014
[1000]	validation_0-rmse:0.86768	validation_1-rmse:25.42853
[1100]	validation_0-rmse:0.69428	validation_1-rmse:25.42353
[1200]	validation_0-rmse:0.58217	validation_1-rmse:25.42213
[1300]	validation_0-rmse:0.50411	validation_1-rmse:25.41850
[1335]	validation_0-rmse:0.48276	validation_1-rmse:25.41909


[I 2026-02-21 20:56:37,592] Trial 1 finished with value: 0.7335574466676273 and parameters: {'n_estimators': 1336, 'max_depth': 10, 'learning_rate': 0.00506253619564565, 'subsample': 0.7683239926922396, 'colsample_bytree': 0.8422207575766958, 'gamma': 1.4676594470465716}. Best is trial 0 with value: 0.7519336313243744.


...

[0]	validation_0-rmse:47.04844	validation_1-rmse:56.90211
[100]	validation_0-rmse:6.30706	validation_1-rmse:24.23962
[200]	validation_0-rmse:3.31801	validation_1-rmse:23.51907
[300]	validation_0-rmse:2.37233	validation_1-rmse:23.37194
[400]	validation_0-rmse:1.78290	validation_1-rmse:23.28876
[500]	validation_0-rmse:1.38965	validation_1-rmse:23.26508
[517]	validation_0-rmse:1.33362	validation_1-rmse:23.26290


[I 2026-02-21 20:59:27,645] Trial 18 finished with value: 0.7768424982973419 and parameters: {'n_estimators': 518, 'max_depth': 5, 'learning_rate': 0.03458884723693423, 'subsample': 0.651158938117497, 'colsample_bytree': 0.7253074392448162, 'gamma': 3.462421259252528}. Best is trial 10 with value: 0.8413577816079185.


[0]	validation_0-rmse:45.21809	validation_1-rmse:55.10524
[100]	validation_0-rmse:3.36693	validation_1-rmse:22.92289
[200]	validation_0-rmse:1.83435	validation_1-rmse:22.83598
[300]	validation_0-rmse:1.18795	validation_1-rmse:22.78023
[400]	validation_0-rmse:0.83105	validation_1-rmse:22.78948
[500]	validation_0-rmse:0.67863	validation_1-rmse:22.78110
[600]	validation_0-rmse:0.65414	validation_1-rmse:22.77841
[700]	validation_0-rmse:0.63301	validation_1-rmse:22.77182
[800]	validation_0-rmse:0.61894	validation_1-rmse:22.77094
[900]	validation_0-rmse:0.61412	validation_1-rmse:22.77173
[1000]	validation_0-rmse:0.60893	validation_1-rmse:22.77346
[1100]	validation_0-rmse:0.60586	validation_1-rmse:22.77393
[1200]	validation_0-rmse:0.60257	validation_1-rmse:22.77438
[1300]	validation_0-rmse:0.59821	validation_1-rmse:22.77428
[1400]	validation_0-rmse:0.59531	validation_1-rmse:22.77433
[1500]	validation_0-rmse:0.59499	validation_1-rmse:22.77430
[1514]	validation_0-rmse:0.59480	validation_1-rmse:

[I 2026-02-21 20:59:36,616] Trial 19 finished with value: 0.7861189454224333 and parameters: {'n_estimators': 1515, 'max_depth': 5, 'learning_rate': 0.07772278216039735, 'subsample': 0.5872451852180509, 'colsample_bytree': 0.6573428662271195, 'gamma': 4.405629866772991}. Best is trial 10 with value: 0.8413577816079185.


Mejor R2 encontrado: 0.84
Mejores parámetros: {'n_estimators': 973, 'max_depth': 3, 'learning_rate': 0.09942910000731134, 'subsample': 0.5462802064328541, 'colsample_bytree': 0.9945829727211198, 'gamma': 4.574448120520625}
